In [1]:
#from config import (census_key, gkey)
import numpy as np
import pandas as pd
import requests
import zillow
# import pprint as pp
import json
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import os
from citipy import citipy
import regex as re

In [2]:
#US top 500 school district rankings from --> https://www.alarms.org/best-school-districts-in-america/#

rankings = pd.read_excel('Data/2020alarms.org_top500schooldist.xlsx')  
rankings.head()

,Rank,State,Name
0,1,Illinois,NAPERVILLE CUSD 203
1,2,Illinois,MORTON CUSD 709
2,3,Massachusetts,LEXINGTON
3,4,Arizona,GILBERT UNIFIED DISTRICT
4,5,Massachusetts,WESTFORD


In [3]:
#Repeat name stripping with Regex
#CONSIDER REMOVING ALL SPACES IN DISTRICT NAME - do for both

rankings['strip_Name'] = rankings['Name'].str.replace(' ', '')
rankings.head()

,Rank,State,Name,strip_Name
0,1,Illinois,NAPERVILLE CUSD 203,NAPERVILLECUSD203
1,2,Illinois,MORTON CUSD 709,MORTONCUSD709
2,3,Massachusetts,LEXINGTON,LEXINGTON
3,4,Arizona,GILBERT UNIFIED DISTRICT,GILBERTUNIFIEDDISTRICT
4,5,Massachusetts,WESTFORD,WESTFORD


In [4]:
rankings['District'] = [re.sub(r'DISTRICT','', i) for i in rankings['strip_Name']]
rankings['District'] = [re.sub(r'SCHOOLS','', i) for i in rankings['District']]
rankings['District'] = [re.sub(r'SCHOOL','', i) for i in rankings['District']]
rankings['District'] = [re.sub(r' S','', i) for i in rankings['District']]
rankings['District'] = [re.sub(r'BOROUGH','', i) for i in rankings['District']]
rankings['District'] = [re.sub(r'PUBLIC','', i) for i in rankings['District']]
rankings.head()

,Rank,State,Name,strip_Name,District
0,1,Illinois,NAPERVILLE CUSD 203,NAPERVILLECUSD203,NAPERVILLECUSD203
1,2,Illinois,MORTON CUSD 709,MORTONCUSD709,MORTONCUSD709
2,3,Massachusetts,LEXINGTON,LEXINGTON,LEXINGTON
3,4,Arizona,GILBERT UNIFIED DISTRICT,GILBERTUNIFIEDDISTRICT,GILBERTUNIFIED
4,5,Massachusetts,WESTFORD,WESTFORD,WESTFORD


In [5]:
rankings[rankings['District'].str.contains("NAPERVILLE")]

,Rank,State,Name,strip_Name,District
0,1,Illinois,NAPERVILLE CUSD 203,NAPERVILLECUSD203,NAPERVILLECUSD203


In [6]:
#US School District data from --> https://catalog.data.gov/dataset/school-district-geographic-relationship-files

districts_zip = pd.read_excel('Data/2015schooldist_byzip.xlsx', encoding='ISO-8859-1',dtype={'ZCTA5CE10': 'str'})  
districts_zip['Zip_code'] = districts_zip['ZCTA5CE10'].apply(lambda x: x.zfill(5))
districts_zip['NAME_LEA15'] = districts_zip['NAME_LEA15'].str.upper() 
districts_zip['strip_name'] = districts_zip['NAME_LEA15'].str.replace(' ', '')

districts_zip.head()

,LEAID,NAME_LEA15,ZCTA5CE10,COUNT,LANDAREA,Zip_code,strip_name
0,100001,FORT RUCKER SCHOOL DISTRICT,00000,6,52.557308,00000,FORTRUCKERSCHOOLDISTRICT
1,100090,ANNISTON CITY SCHOOL DISTRICT,00000,6,12.844220,00000,ANNISTONCITYSCHOOLDISTRICT
2,100188,CHICKASAW CITY SCHOOL DISTRICT,00000,4,0.000000,00000,CHICKASAWCITYSCHOOLDISTRICT
3,100270,BALDWIN COUNTY SCHOOL DISTRICT,00000,25,135.539762,00000,BALDWINCOUNTYSCHOOLDISTRICT
4,100360,BIBB COUNTY SCHOOL DISTRICT,00000,13,53.863187,00000,BIBBCOUNTYSCHOOLDISTRICT


In [7]:
#NEED to figure out how to remove both 'SCHOOL' and 'SCHOOLS'

districts_zip['District'] = [re.sub(r'DISTRICT','', i) for i in districts_zip['strip_name']]
districts_zip['District'] = [re.sub(r'SCHOOLS','', i) for i in districts_zip['District']]
districts_zip['District'] = [re.sub(r'SCHOOL','', i) for i in districts_zip['District']]
districts_zip['District'] = [re.sub(r'BOROUGH','', i) for i in districts_zip['District']]
districts_zip['District'] = [re.sub(r'PUBLIC','', i) for i in districts_zip['District']]
districts_zip.drop(districts_zip[districts_zip['ZCTA5CE10'] == '00000'].index, inplace = True) 
districts_zip.head()

,LEAID,NAME_LEA15,ZCTA5CE10,COUNT,LANDAREA,Zip_code,strip_name,District
2343,7200030,PUERTO RICO DEPARTMENT OF EDUCATION,00601,132,64.347743,00601,PUERTORICODEPARTMENTOFEDUCATION,PUERTORICODEPARTMENTOFEDUCATION
2344,7200030,PUERTO RICO DEPARTMENT OF EDUCATION,00602,132,30.612962,00602,PUERTORICODEPARTMENTOFEDUCATION,PUERTORICODEPARTMENTOFEDUCATION
2345,7200030,PUERTO RICO DEPARTMENT OF EDUCATION,00603,132,31.615793,00603,PUERTORICODEPARTMENTOFEDUCATION,PUERTORICODEPARTMENTOFEDUCATION
2346,7200030,PUERTO RICO DEPARTMENT OF EDUCATION,00606,132,42.309076,00606,PUERTORICODEPARTMENTOFEDUCATION,PUERTORICODEPARTMENTOFEDUCATION
2347,7200030,PUERTO RICO DEPARTMENT OF EDUCATION,00610,132,35.915722,00610,PUERTORICODEPARTMENTOFEDUCATION,PUERTORICODEPARTMENTOFEDUCATION


In [8]:

merged_df = districts_zip.merge(rankings, on='District', how='outer').drop(columns=['strip_name', 'strip_Name'])

merged_df.head()

,LEAID,NAME_LEA15,ZCTA5CE10,COUNT,LANDAREA,Zip_code,District,Rank,State,Name
0,7200030.0,PUERTO RICO DEPARTMENT OF EDUCATION,00601,132.0,64.347743,00601,PUERTORICODEPARTMENTOFEDUCATION,NaN,NaN,NaN
1,7200030.0,PUERTO RICO DEPARTMENT OF EDUCATION,00602,132.0,30.612962,00602,PUERTORICODEPARTMENTOFEDUCATION,NaN,NaN,NaN
2,7200030.0,PUERTO RICO DEPARTMENT OF EDUCATION,00603,132.0,31.615793,00603,PUERTORICODEPARTMENTOFEDUCATION,NaN,NaN,NaN
3,7200030.0,PUERTO RICO DEPARTMENT OF EDUCATION,00606,132.0,42.309076,00606,PUERTORICODEPARTMENTOFEDUCATION,NaN,NaN,NaN
4,7200030.0,PUERTO RICO DEPARTMENT OF EDUCATION,00610,132.0,35.915722,00610,PUERTORICODEPARTMENTOFEDUCATION,NaN,NaN,NaN


In [9]:
merged_df[merged_df['District'].str.contains("TENAFLY")]

,LEAID,NAME_LEA15,ZCTA5CE10,COUNT,LANDAREA,Zip_code,District,Rank,State,Name
5435,3416110.0,TENAFLY BOROUGH SCHOOL DISTRICT,07670,1.0,4.6008,07670,TENAFLY,11.0,New Jersey,TENAFLY PUBLIC SCHOOLS


In [14]:
index = merged_df.index
number_of_rows = len(index)
number_of_rows

82207

In [17]:
clean_merged_df = merged_df[merged_df['Rank'].notna()]
clean_merged_df.to_csv('rankings.csv')

Plan B

In [11]:
# from fuzzywuzzy import fuzz
# from fuzzywuzzy import process

# def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
#     """
#     :param df_1: the left table to join
#     :param df_2: the right table to join
#     :param key1: key column of the left table
#     :param key2: key column of the right table
#     :param threshold: how close the matches should be to return a match, based on Levenshtein distance
#     :param limit: the amount of matches that will get returned, these are sorted high to low
#     :return: dataframe with boths keys and matches
#     """
#     s = df_2[key2].tolist()

#     m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
#     df_1['matches'] = m

#     m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
#     df_1['matches'] = m2

#     return df_1

In [12]:
# fuzzy_merge(districts_zip, rankings, 'District', 'District', threshold=80)